In [ ]:
import sys
import torch
import matplotlib.pyplot as plt
from captum.attr import Saliency, GuidedGradCam, DeepLift, Lime, DeepLiftShap

sys.path.append("..")
from lib import models, utils

In [ ]:
device = utils.select_device()

generator = models.Generator28(noise_dim=100, channels=1, feature_maps=64).to(device)
discriminator = models.Discriminator28(channels=1, feature_maps=64).to(device)

generator.load_state_dict(torch.load('../weights/dcgan/mnist/gen_epoch_10.pth', map_location=torch.device(device)))
discriminator.load_state_dict(torch.load('../weights/dcgan/mnist/disc_epoch_10.pth', map_location=torch.device(device)))

In [ ]:
noise = torch.randn(1, 100, 1, 1, device=device)
fake = generator(noise)

# Saliency
saliency = Saliency(discriminator)
s_exp = saliency.attribute(fake)

# DeepLift
dl = DeepLift(discriminator)
dl_exp = dl.attribute(fake)

# GuidedGradCam
ggc = GuidedGradCam(discriminator, discriminator.network[9])
ggc_exp = ggc.attribute(fake)

# Lime
lime = Lime(discriminator)
lime_exp = lime.attribute(fake, n_samples=200)

# DeepLiftShap
dls = DeepLiftShap(discriminator)
dls_exp = dls.attribute(fake)

In [ ]:
plt.figure(figsize=(7, 7))
plt.subplot(1, 6, 1)
plt.imshow(fake.detach().squeeze().numpy(), cmap=plt.cm.gray)
plt.title("Original")
plt.xticks([])
plt.yticks([])

plt.subplot(1, 6, 2)
plt.imshow(s_exp.detach().squeeze().numpy(), cmap=plt.cm.jet)
plt.title("Saliency")
plt.xticks([])
plt.yticks([])

plt.subplot(1, 6, 3)
plt.imshow(dl_exp.detach().squeeze().numpy(), cmap=plt.cm.jet)
plt.title("DeepLift")
plt.xticks([])
plt.yticks([])

plt.subplot(1, 6, 4)
plt.imshow(ggc_exp.cpu().detach().squeeze().numpy(), cmap=plt.cm.jet)
plt.title("GuidedGradCam")
plt.xticks([])
plt.yticks([])

plt.subplot(1, 6, 5)
plt.imshow(lime_exp.cpu().detach().squeeze().numpy(), cmap=plt.cm.jet)
plt.title("Lime")
plt.xticks([])
plt.yticks([])

plt.subplot(1, 6, 4)
plt.imshow(dls_exp.cpu().detach().squeeze().numpy(), cmap=plt.cm.jet)
plt.title("DeepLiftShap")
plt.xticks([])
plt.yticks([])